In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
! pip install seaborn

# Load Data

In [ ]:
source = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

In [ ]:
data = pd.read_csv(
    source,
    names=column_names, na_values='?',
    sep=' ', skipinitialspace=True,
    comment='\t'
    )

In [ ]:
dataCopy = data.copy()

In [ ]:
data.head()

In [ ]:
data.info()

# Pre-Process Data

In [ ]:
data.dropna(axis=0, inplace=True)

In [ ]:
data.Origin.value_counts()

In [ ]:
data.Origin.map({1:"USA", 2:"EUROPE", 3:"JAPAN"})

In [ ]:
data = pd.get_dummies(data, columns=['Origin'])
data

# Exploratory Data Analysis

In [ ]:
data.describe().T

In [ ]:
sns.pairplot(data, palette='Set2')

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(data.corr(), annot=True)

In [ ]:
sns.boxplot(x=data.Horsepower)

In [ ]:
q1 = np.percentile(data.Horsepower, 25)
q3 = np.percentile(data.Horsepower, 75)
iqr = q3 - q1
iqr

In [ ]:
outliers_index_lower = np.where(data.Horsepower < q1 - (1.5 * iqr))
outliers_index_lower

In [ ]:
outliers_index_upper = np.where(data.Horsepower > q3 + (1.5 * iqr))
outliers_index_upper[0]

In [ ]:
data.drop(outliers_index_upper[0], inplace=True)

In [ ]:
data

# Train Test Set

In [2]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(
    data[['Horsepower']], data[['MPG']], test_size=0.2, random_state=42
)

NameError: name 'data' is not defined

# Neural Network Modeling

In [ ]:
normalizer = tf.keras.layers.Normalization(input_shape=[1,], axis=-1)
normalizer.adapt(xtrain)

## Single Neuron

In [ ]:
# Defining The Structure
model_single_neuron = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units=1, activation=tf.keras.activations.relu)
])

In [ ]:
# Defining Optimization
model_single_neuron.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    loss=tf.keras.losses.MSE,
    metrics=['mse', 'mae']
)

In [ ]:
model_single_neuron.fit(
    x=xtrain, y=ytrain,
    epochs=1000, verbose=2,
    validation_split=0.2
)

In [ ]:
model_single_neuron.summary()

In [ ]:
history = pd.DataFrame(model_single_neuron.history.history)
history

## Shallow Neural Network

In [ ]:
# Defining The Structure
shallowNN = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(units=1, activation=tf.keras.activations.relu)
])

In [ ]:
# Defining Optimization
shallowNN .compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    loss=tf.keras.losses.MSE,
    metrics=['mse']
)

In [ ]:
shallowNN .fit(
    x=xtrain, y=ytrain,
    epochs=100, verbose=2,
    validation_split=0.2
)

In [ ]:
shallowNN.summary()

## Deep Neural Network

In [ ]:
# Defining The Structure
deepNN = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(units=28, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(units=1, activation=tf.keras.activations.relu)
])

In [ ]:
# Defining Optimization
deepNN.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    loss=tf.keras.losses.MSE,
    metrics=['mse']
)

In [ ]:
deepNN.fit(
    x=xtrain, y=ytrain,
    epochs=100, verbose=2,
    validation_split=0.2
)

In [ ]:
deepNN.summary()

# Testing and Evaluation